In [60]:
#cargar paquetes
import pandas as pd 
import sqlite3 as sql 
#import a_funciones as funciones  
import sys 
from sklearn.impute import SimpleImputer

In [266]:
#funciones, despues se pasan a un archivo aparte
def impute_columns(df, columns, strategy): #Función que imputa datos para variables numéricas
  imputer = SimpleImputer(strategy=strategy)
  for column in columns:
    column_imputed = imputer.fit_transform(df[column].values.reshape(-1, 1))
    df[column] = column_imputed.flatten()
  return df
def categorias(pandas_db,sql_db):
    """pandas_db= Base de datos con el df de pandas.\n
       sql_db= nombre de la base de datos entre comillas "" de sql declarada en el .to_sql() """
    for i in range(pandas_db.shape[1]):
        db_name=sql_db
        column_name=pandas_db.columns.values[i]
        texto= f'select {column_name}, count(*) as "qty" from {db_name} group by {column_name}'   
        print(f"****Variable {column_name}****")
        print(pd.read_sql(texto, conn))

In [274]:
#cargar bases de datos
df_employee_survey=pd.read_csv("/Users/Vlado/Documents/universidad/11_sem/Analitica3/caso_estudio_HR/employee_survey_data.csv")
df_general=pd.read_csv("/Users/Vlado/Documents/universidad/11_sem/Analitica3/caso_estudio_HR/general_data.csv",sep = ';')
df_manager=pd.read_csv("/Users/Vlado/Documents/universidad/11_sem/Analitica3/caso_estudio_HR/manager_survey_data.csv")
df_retirement=pd.read_csv("/Users/Vlado/Documents/universidad/11_sem/Analitica3/caso_estudio_HR/retirement_info.csv",sep = ';')
df_out_time=pd.read_csv("/Users/Vlado/Documents/universidad/11_sem/Analitica3/caso_estudio_HR/out_time.csv")
df_in_time=pd.read_csv("/Users/Vlado/Documents/universidad/11_sem/Analitica3/caso_estudio_HR/in_time.csv")



##Vista preliminar de las bases,nulos, categorias y outliners

In [261]:
### llevar bases de datos a sql

conn= sql.connect("db_empleados") ### crea una base de datos con el nombre dentro de comillas, si existe crea una conexión.

### mostrar tablas dentro de la base de datos
cursor=conn.cursor() 
cursor.execute("select name from sqlite_master where type='table'")
cursor.fetchall()

[('general',), ('manager',), ('employee_survey',), ('retirement',)]

In [275]:
#tabla employe_survey
print(df_employee_survey.info())
df_employee_survey.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 137.9 KB
None


,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


In [276]:
#TRATO DE NULOS CON MEDIANA
df_employee_survey =impute_columns(df_employee_survey, ['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance'], 'median')
df_employee_survey.isnull().sum()

EmployeeID                 0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
dtype: int64

In [267]:
### Llevar tablas a base de datos sql
df_employee_survey.to_sql("employee_survey",conn,if_exists="replace")

##ver categorias
categorias(df_employee_survey,"employee_survey")

****Variable EmployeeID****
      EmployeeID  qty
0              1    1
1              2    1
2              3    1
3              4    1
4              5    1
...          ...  ...
4405        4406    1
4406        4407    1
4407        4408    1
4408        4409    1
4409        4410    1

[4410 rows x 2 columns]
****Variable EnvironmentSatisfaction****
   EnvironmentSatisfaction   qty
0                      1.0   845
1                      2.0   856
2                      3.0  1375
3                      4.0  1334
****Variable JobSatisfaction****
   JobSatisfaction   qty
0              1.0   860
1              2.0   840
2              3.0  1343
3              4.0  1367
****Variable WorkLifeBalance****
   WorkLifeBalance   qty
0              1.0   239
1              2.0  1019
2              3.0  2698
3              4.0   454


In [28]:
#tabla general data
print(df_general.info())
df_general.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   BusinessTravel           4410 non-null   object 
 2   Department               4410 non-null   object 
 3   DistanceFromHome         4410 non-null   int64  
 4   Education                4410 non-null   int64  
 5   EducationField           4410 non-null   object 
 6   EmployeeCount            4410 non-null   int64  
 7   EmployeeID               4410 non-null   int64  
 8   Gender                   4410 non-null   object 
 9   JobLevel                 4410 non-null   int64  
 10  JobRole                  4410 non-null   object 
 11  MaritalStatus            4410 non-null   object 
 12  MonthlyIncome            4410 non-null   int64  
 13  NumCompaniesWorked       4391 non-null   float64
 14  Over18                  

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,3,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,1,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [269]:
#TRATO DE NULOS CON MEDIANA
df_general=impute_columns(df = df_general, columns = ['NumCompaniesWorked', 'TotalWorkingYears'], strategy = 'median')
df_general.isnull().sum()

Age                        0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
EmployeeID                 0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
Over18                     0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
dtype: int64

In [271]:
### Llevar tablas a base de datos sql
df_general.to_sql("general",conn,if_exists="replace")
##ver categorias
categorias(df_general,"general")

****Variable Age****
    Age  qty
0    18   24
1    19   27
2    20   33
3    21   39
4    22   48
5    23   42
6    24   78
7    25   78
8    26  117
9    27  144
10   28  144
11   29  204
12   30  180
13   31  207
14   32  183
15   33  174
16   34  231
17   35  234
18   36  207
19   37  150
20   38  174
21   39  126
22   40  171
23   41  120
24   42  138
25   43   96
26   44   99
27   45  123
28   46   99
29   47   72
30   48   57
31   49   72
32   50   90
33   51   57
34   52   54
35   53   57
36   54   54
37   55   66
38   56   42
39   57   12
40   58   42
41   59   30
42   60   15
****Variable BusinessTravel****
      BusinessTravel   qty
0         Non-Travel   450
1  Travel_Frequently   831
2      Travel_Rarely  3129
****Variable Department****
               Department   qty
0         Human Resources   189
1  Research & Development  2883
2                   Sales  1338
****Variable DistanceFromHome****
    DistanceFromHome  qty
0                  1  624
1                  2  633

In [29]:
#tabla manager survey
print(df_manager.info())
df_manager.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   EmployeeID         4410 non-null   int64
 1   JobInvolvement     4410 non-null   int64
 2   PerformanceRating  4410 non-null   int64
dtypes: int64(3)
memory usage: 103.5 KB
None


,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


In [272]:
### Llevar tablas a base de datos sql
df_manager.to_sql("manager",conn,if_exists="replace")
##ver categorias
categorias(df_manager,"manager")

****Variable EmployeeID****
      EmployeeID  qty
0              1    1
1              2    1
2              3    1
3              4    1
4              5    1
...          ...  ...
4405        4406    1
4406        4407    1
4407        4408    1
4408        4409    1
4409        4410    1

[4410 rows x 2 columns]
****Variable JobInvolvement****
   JobInvolvement   qty
0               1   249
1               2  1125
2               3  2604
3               4   432
****Variable PerformanceRating****
   PerformanceRating   qty
0                  3  3732
1                  4   678


In [32]:
#tabla retirement info
print(df_retirement.info())
df_retirement.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         711 non-null    int64 
 1   Attrition          711 non-null    object
 2   retirementDate     711 non-null    object
 3   retirementType     711 non-null    object
 4   resignationReason  641 non-null    object
dtypes: int64(1), object(4)
memory usage: 27.9+ KB
None


,EmployeeID,Attrition,retirementDate,retirementType,resignationReason
0,2,Yes,15/10/2016,Resignation,Others
1,7,Yes,20/07/2016,Resignation,Stress
2,14,Yes,08/11/2016,Resignation,Others
3,29,Yes,15/10/2016,Resignation,Others
4,31,Yes,08/02/2016,Resignation,Salary


In [273]:
### Llevar tablas a base de datos sql
df_retirement.to_sql("retirement",conn,if_exists="replace")

##ver categorias
categorias(df_retirement,"retirement")


****Variable EmployeeID****
     EmployeeID  qty
0             2    1
1             7    1
2            14    1
3            29    1
4            31    1
..          ...  ...
706        4382    1
707        4387    1
708        4389    1
709        4392    1
710        4403    1

[711 rows x 2 columns]
****Variable Attrition****
  Attrition  qty
0       Yes  711
****Variable retirementDate****
    retirementDate  qty
0       01/02/2016    2
1       01/03/2016    2
2       01/04/2016    1
3       01/05/2016    4
4       01/06/2016    6
..             ...  ...
303     31/01/2016    4
304     31/03/2016    3
305     31/07/2016    3
306     31/08/2016    2
307     31/10/2016    5

[308 rows x 2 columns]
****Variable retirementType****
  retirementType  qty
0          Fired   70
1    Resignation  641
****Variable resignationReason****
  resignationReason  qty
0              None   70
1            Others  323
2            Salary  189
3            Stress  129
